In [ ]:
%load_ext autoreload
%autoreload 3

In [ ]:
import os
import sys

sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(""))))

# Auto Machine Learning

Scikit-learn nos permite semi-automatizar la optimización de nuestros modelos con GridSearchCV y RandomizedSearchCV, ¿Cuál es el límite de esta automatización? Haciendonos esta pregunta nace un nuevo concepto Automated Machine Learning.

Automated Machine Learning (AutoML), es un concepto que en general pretende la completa automatización de todo el proceso de Machine Learning, desde la extracción de los datos hasta su publicación final de cara a los usuarios.

Sin embargo, este ideal aún está en desarrollo en la mayoría de las etapas del proceso de Machine Learning y aún se depende bastante de la intervención humana.

odemos encontrar más información leyendo el siguiente enlace: [Qué es Automated Machine Learning: la próxima generación de inteligencia artificial](https://www.itmastersmag.com/noticias-analisis/que-es-automated-machine-learning-la-proxima-generacion-de-inteligencia-artificial/)

Existe una implementación de este concepto utilizando Scikit Learn llamado auto-sklearn. Esto nos ayudará a llevar un paso más lejos nuestro proceso de selección y optimización de modelos de machine learning. Dado que automáticamente prueba diferentes modelos predefinidos y configuraciones de parámetros comunes hasta encontrar la que más se ajuste según los datos que le pasemos como entrada. Con esta herramienta podrás entrenar modelos tanto de clasificación como de regresión por igual.

- [Lista de los clasificadores disponibles](https://github.com/automl/auto-sklearn/tree/master/autosklearn/pipeline/components/classification)

- [Lista de los regresores disponibles](https://github.com/automl/auto-sklearn/tree/master/autosklearn/pipeline/components/regression)

Se puede añadir modelos personalizados al proceso siguiendo los pasos descritos en la documentación.

## Auto-sklearn

Esta herramienta es una librería basada en los algoritmos de scikit-learn, aunque hay que tener presente que es una librería externa y se debe instalar siempre por aparte. En todo caso al ser una librería de Python se puede combinar sin ningún problema con el resto de nuestro código desarrollado para scikit-learn, incluso permitiendo la exportación de modelos ya entrenados para su posterior uso.

[Auto-Sklearn](https://automl.github.io/auto-sklearn/master/index.html)

La forma de hacer funcionar nuestro algoritmo no podría ser más fácil. Nos resultará bastante familiar a estas alturas después de haber trabajado tanto con sklearn.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autosklearn.classification import AutoSklearnClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from pprint import pprints

In [ ]:
df = pd.read_csv("../datasets/heart.csv")
data = df.drop(columns=["target"])
target = df["target"]
x_train, x_test, y_train, y_test = train_test_split(
    data, target, test_size=0.3, random_state=17
)
print("Instanciando clasificador")
cls = AutoSklearnClassifier(
    time_left_for_this_task=120, n_jobs=4, seed=5, per_run_time_limit=30
)
print("Entrenando modelo")
cls.fit(x_train, y_train)
print("Realizando predicciones")
y_pred = cls.predict(x_test)
score = roc_auc_score(y_test, y_pred)
# View the models found by auto-sklearn
print(cls.leaderboard())
# Print the final ensamble constructed by auto-sklearn
pprint(cls.show_models(), indent=4)
acc = accuracy_score(y_test, y_pred)
print("Accuracy score:", acc)
print("AUC:", score)

Valor esperado:

```python
          rank  ensemble_weight                 type      cost   duration
model_id
2            1             0.04        random_forest  0.042194   1.575469
18           3             0.02    gradient_boosting  0.042194   1.533511
21           2             0.04          extra_trees  0.042194   1.574728
47           4             0.02    gradient_boosting  0.050633   2.261632
11           5             0.02    gradient_boosting  0.054852   1.173174
33           6             0.02  k_nearest_neighbors  0.054852   1.879181
3            7             0.04    gradient_boosting  0.059072   1.418526
15           8             0.16                  mlp  0.063291   3.093189
16           9             0.02          extra_trees  0.067511   2.205926
13          10             0.02        random_forest  0.088608  10.945232
12          11             0.02             adaboost  0.113924   1.787503
39          12             0.02    gradient_boosting  0.113924   1.992045
14          13             0.02                  mlp  0.147679   1.898323
5           14             0.04             adaboost  0.164557   1.426467
6           16             0.16   passive_aggressive  0.172996   0.828161
31          15             0.08             adaboost  0.172996   2.799388
37          17             0.06   passive_aggressive  0.202532   2.236975
24          18             0.10          gaussian_nb  0.206751   0.841711
41          19             0.10                  lda  0.316456   1.652267
```


Ejemplo de configuracion de clasificadores encontrados:

```python
{
    2: {   'balancing': Balancing(random_state=5),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fbd7f202650>,
           'cost': 0.04219409282700426,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fbd7f213dc0>,
           'ensemble_weight': 0.04,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fbd7f2018d0>,
           'model_id': 2,
           'rank': 1,
           'sklearn_classifier': RandomForestClassifier(max_features=3, n_estimators=512, n_jobs=1,
                       random_state=5, warm_start=True)},
    3: {   'balancing': Balancing(random_state=5, strategy='weighting'),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fbd67873100>,
           'cost': 0.05907172995780585,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fbd80475060>,
           'ensemble_weight': 0.04,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fbd678715a0>,
           'model_id': 3,
           'rank': 2,
           'sklearn_classifier': HistGradientBoostingClassifier(early_stopping=False,
                               l2_regularization=4.821686883442146e-05,
                               learning_rate=0.10161621495242192, max_iter=512,
                               max_leaf_nodes=535, min_samples_leaf=10,
                               n_iter_no_change=0, random_state=5,
                               validation_fraction=None, warm_start=True)}
}
Accuracy score: 0.9837662337662337
AUC: 0.9839285714285715
```
